In [79]:
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
from pathlib import Path
#from tensorflow.keras.layers import BatchNormalization
#from tensorflow.keras.layers import Layer, InputSpec
from imageai.Detection import ObjectDetection
from imageai.Detection.Custom import DetectionModelTrainer
from imageai.Detection.Custom import CustomObjectDetection

from PIL import Image

import albumentations as A
import cv2

import xml.etree.ElementTree as ET


In [80]:
train_path = 'data_dz7/train_zip/train/'
test_path = 'data_dz7/test_zip/test/'

annots_train_path = sorted([i for i in Path(train_path).glob('*.xml')])
images_train_path = sorted([i for i in Path(train_path).glob('*.jpg')])

annots_test_path = sorted([i for i in Path(test_path).glob('*.xml')])
images_test_path = sorted([i for i in Path(test_path).glob('*.xml')])

classes = np.array(["Apple", 'Banana', 'Orange'])

n_imgs = len(images_train_path)
print(f'train - {n_imgs}, test -{len(images_test_path)}')

train - 240, test -60


In [81]:
def editXML(file):
    
    my_file = open(file, 'r')
    string = my_file.read()
    
    width = string[string.find('<width>')+len('<width>') : string.find('</width>')]
    height = string[string.find('<height>')+len('<height>') : string.find('</height>')]
    
    if width == '0' or height == '0':
        
        folder = string[string.find('<folder>')+len('<folder>') : string.find('</folder>')]
        filename = string[string.find('<filename>')+len('<filename>') : string.find('</filename>')]
        path_img = f'data_dz7/{folder}_zip/{folder}/{filename}'
        
        im = Image.open(path_img)
        (width_im, height_im) = im.size

        tree = ET.parse(file)
        root = tree.getroot()

        for elem in root.iter("width"):
            elem.text = str(width_im)

        for elem in root.iter("height"):
            elem.text = str(height_im)

        tree.write('newfile.xml')

        return 'newfile.xml'
    
    else:
        return file

In [82]:
def augmentation_image_and_XML(file):

    my_file = open(file, 'r')
    string = my_file.read()

    path_image = file.replace('annotations', 'images')
    image = cv2.imread(path_image[:-3]+'jpg')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    bboxes = []
    class_labels = []
    
    for item in string.split('</object>'):
        if '<object>' in item:
            bboxes.append([int(item[item.find('<xmin>')+len('<xmin>') : item.find('</xmin>')]),
                           int(item[item.find('<ymin>')+len('<ymin>') : item.find('</ymin>')]),
                           int(item[item.find('<xmax>')+len('<xmax>') : item.find('</xmax>')]),
                           int(item[item.find('<ymax>')+len('<ymax>') : item.find('</ymax>')])])
            class_labels.append(item[item.find('<name>')+len('<name>') : item.find('</name>')])

    transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.RandomBrightnessContrast(p=0.2),
    A.ChannelDropout(channel_drop_range=(1, 1), fill_value=0, p=0.1),
    ], bbox_params=A.BboxParams(format='pascal_voc', min_area=1024, min_visibility=0.1, label_fields=['class_labels']))
    
    for i in range(1, 5):
        transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        transformed_image = transformed['image']
        transformed_bboxes = transformed['bboxes']
        transformed_class_labels = transformed['class_labels']

        # сохранить изображение
        Image.fromarray(transformed_image).save(f'{path_image[:-4]}-{i}.jpg',quality=95)
        
        # сохранить аннотацию 
        folder = string[string.find('<folder>')+len('<folder>') : string.find('</folder>')]
        filename = string[string.find('<filename>')+len('<filename>') : string.find('</filename>')]
        width = string[string.find('<width>')+len('<width>') : string.find('</width>')]
        height = string[string.find('<height>')+len('<height>') : string.find('</height>')]
        
        annotation = ET.Element("annotation")
        ET.SubElement(annotation, "folder").text = folder
        ET.SubElement(annotation, "filename").text = str(f'{filename[:-4]}-{i}.jpg')
        ET.SubElement(annotation, "path").text = str(f'{path_image[:-4]}-{i}.jpg')
        source = ET.SubElement(annotation, "source")
        ET.SubElement(source, "database").text = "Unknown"
        size = ET.SubElement(annotation, "size")
        ET.SubElement(size, "width").text = str(width)
        ET.SubElement(size, "height").text = str(height)
        ET.SubElement(size, "depth").text = "3"
        ET.SubElement(annotation, "segmented").text = "0"
        for num, box in enumerate(transformed_bboxes):
            object = ET.SubElement(annotation, "object")
            ET.SubElement(object, "name").text = transformed_class_labels[num]
            ET.SubElement(object, "pose").text = "Unspecified"
            ET.SubElement(object, "truncated").text = "0"
            ET.SubElement(object, "difficult").text = "0"
            bndbox = ET.SubElement(object, "bndbox")
            ET.SubElement(bndbox, "xmin").text = str(int(box[0])) 
            ET.SubElement(bndbox, "ymin").text = str(int(box[1]))
            ET.SubElement(bndbox, "xmax").text = str(int(box[2]))
            ET.SubElement(bndbox, "ymax").text = str(int(box[3]))

        tree = ET.ElementTree(annotation)
        tree.write(f'{file[:-4]}-{i}.xml')

In [83]:
# создание директорий для сохранение предобработанных данных
os.makedirs('imageai/data/train/images', exist_ok=True)
os.makedirs('imageai/data/train/annotations', exist_ok=True)

os.makedirs('imageai/data/validation/images', exist_ok=True)
os.makedirs('imageai/data/validation/annotations', exist_ok=True)

os.makedirs('imageai/data/test/images', exist_ok=True)
os.makedirs('imageai/data/test/annotations', exist_ok=True)

In [84]:
# формирование списка классов
set_classes = set()
annots_path = annots_train_path + annots_test_path

for annot in annots_path:
    with open(annot, 'r') as f:
        string = f.read()
        name = string[string.find('<name>')+6:string.find('</name>')]
        set_classes.add(name)

classes = np.array(list(set_classes))
print(classes)

['apple' 'orange' 'banana']


In [85]:
for i, (annot_path, img_path) in enumerate(zip(annots_train_path, images_train_path)):   
    if i > n_imgs:
        break
        
    if i%5 == 0:
        new_path = shutil.copy(img_path, 'imageai/data/validation/images/' + img_path.parts[-1])
        # редактирование валидационных данных
        annot_path_edit = editXML(annot_path)
        annot_path_new = shutil.copy(annot_path_edit, 'imageai/data/validation/annotations/' + annot_path.parts[-1])
        # аугментация валидационных данных
        augmentation_image_and_XML(annot_path_new)
    else:
        shutil.copy(img_path, 'imageai/data/train/images/' + img_path.parts[-1])
        # редактирование тренировочных данных
        annot_path_edit = editXML(annot_path)
        annot_path_new = shutil.copy(annot_path_edit, 'imageai/data/train/annotations/' + annot_path.parts[-1])
        # аугментация тренировочных данных
        augmentation_image_and_XML(annot_path_new)
        
for i, (annot_test_path, img_test_path) in enumerate(zip(annots_test_path, images_test_path)):  
    shutil.copy(img_test_path, 'imageai/data/test/images/' + img_test_path.parts[-1])
    annot_test_path_new = editXML(annot_test_path)
    shutil.copy(annot_test_path_new, 'imageai/data/test/annotations/' + annot_test_path.parts[-1])

In [86]:
detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath("data_dz7/yolo.h5")
detector.loadModel()
detections = detector.detectObjectsFromImage(input_image='data_dz7/train_zip/train/apple_31.jpg',
                                             output_image_path='detected.jpg',
                                             minimum_percentage_probability=50)

for eachObject in detections:
    print(eachObject["name"] , " : ", eachObject["percentage_probability"], " : ", eachObject["box_points"] )
    print("--------------------------------")

ValueError: Subshape must have computed start >= end since stride is negative, but is 0 and 2 (computed from start 0 and end 9223372036854775807 over shape with rank 2 and stride-1)